In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

import matplotlib.pyplot as plt 
import numpy as np
import random
import json
import time

import math
from models import LogisticRegressor, Convolutional

In [2]:
N_EPOCHS = 10000
N_FEATURES = 12
W_DIM = 4

In [14]:
data = []

n_true, n_false = 0, 0

def construct_tensors(vector, state):
    vector = np.array(vector[:N_FEATURES]).reshape(N_FEATURES * W_DIM)
    norm = np.linalg.norm(vector)
    x0 = torch.FloatTensor(vector / norm)
    x1 = torch.FloatTensor([vector / norm])
    y = torch.tensor(state)
    return (x0, x1, y)

with open('data/mfccs.json') as f:
    json_data = json.load(f)

    # Loop over all videos in dataset
    for video in json_data:
        video_data = json_data[video]

        # Loop over all bounces for current video
        for feature_vector in video_data['true']:
            if feature_vector != []:
                data.append(construct_tensors(feature_vector, 1))
                n_true += 1

        for feature_vector in video_data['false']:
            if feature_vector != []:
                data.append(construct_tensors(feature_vector, 0))
                n_false += 1

print(f'Extracted {len(data)} datapoints ({n_true} true and {n_false} false)')
print(f'n_features={data[0][0].shape}')

ValueError: cannot reshape array of size 512 into shape (48,)

In [4]:
# Split dataset into train and test (85% train)
np.random.shuffle(data)
split_ind = int(len(data) * 0.80)
train_data = data[:split_ind]
test_data = data[split_ind:]

In [5]:
train_loader = torch.utils.data.DataLoader(dataset=train_data,
                                           batch_size=128,
                                           shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_data,
                                           batch_size=128,
                                           shuffle=True)

In [6]:
device = torch.device("cuda:0" if (torch.cuda.is_available()) else "cpu")
print(f'Device: {device}')

Device: cpu


In [7]:
regressor, conv = LogisticRegressor(W_DIM, N_FEATURES, 2).to(device), Convolutional(N_FEATURES, 2).to(device)
print(f'n_params={sum(p.numel() for p in regressor.parameters())}')
print(f'n_params={sum(p.numel() for p in conv.parameters())}')

n_params=98
n_params=442


In [8]:
optis = [optim.SGD(regressor.parameters(), lr=0.01), optim.SGD(conv.parameters(), lr=0.01)]
crits = [nn.CrossEntropyLoss(), nn.CrossEntropyLoss()]

In [9]:
def accuracy(model, data, inds):
    x, y = inds
    correct, total = 0, 0
    for xxy in data:
        y_hat = model(xxy[x].to(device))
        _, predicted = torch.max(y_hat.data, 1)
        
        total += xxy[y].size(0)
        correct += (predicted.cpu() == xxy[y]).sum()

    return 100 * int(correct) / total, (predicted.cpu() != xxy[y])

def confusion(model, data, inds):
    x, y = inds
    matrix = np.array([[0, 0], [0, 0]])
    exampl = [[[], []], [], []]
    for xxy in data:
        y_hat = model(xxy[x].to(device))
        _, predicted = torch.max(y_hat.data, 1)
        predicted = predicted.cpu()

        matrix[0, 0] += torch.sum((xxy[y] == 1) & (predicted == 1))
        matrix[0, 1] += torch.sum((xxy[y] == 0) & (predicted == 1))
        matrix[1, 0] += torch.sum((xxy[y] == 1) & (predicted == 0))
        matrix[1, 1] += torch.sum((xxy[y] == 0) & (predicted == 0))

    return matrix, []

def recall(model, data, inds):
    x, y = inds
    tp, fn = 0, 0
    for xxy in data:
        y_hat = model(xxy[x].to(device))
        _, predicted = torch.max(y_hat.data, 1)

        tp += torch.sum((xxy[y] == 1) & (predicted.cpu() == 1))
        fn += torch.sum((xxy[y] == 1) & (predicted.cpu() == 0))
    return int(tp) / (int(tp) + int(fn))


In [10]:
accuracy_histories = [[], []]
n_iters = 0
start = time.time()

for epoch in range(1, N_EPOCHS + 1):
    np.random.shuffle(data)
    for i, (x0, x1, y) in enumerate(train_loader):
        # Clear gradients
        for opti in optis:
            opti.zero_grad()

        # Make prediction and calculate loss
        y_hat0 = regressor(x0.to(device)).cpu()
        y_hat1 = conv(x1.to(device)).cpu()

        loss0 = crits[0](y_hat0, y)
        loss1 = crits[1](y_hat1, y)

        # Calculate new gradients and optimize
        loss0.backward()
        loss1.backward()
        for opti in optis:
            opti.step()

        if n_iters % 250 == 0 or epoch == N_EPOCHS:
            acc0, _ = accuracy(regressor, test_loader, (0, 2))
            acc1, _ = accuracy(conv, test_loader, (1, 2))

            accuracy_histories[0].append(acc0)
            accuracy_histories[1].append(acc1)
            
            fepoch = 'Epoch: \033[92m{}\033[0m/\033[92m{}\033[0m'.format(epoch, N_EPOCHS)
            fiters = 'Iter: \033[92m{}\033[0m'.format(n_iters)
            facc0 = 'Acc0: \033[92m{:.2f}%\033[0m'.format(acc0)
            facc1 = 'Acc1: \033[92m{:.2f}%\033[0m'.format(acc1)
            fptime = 'Time: \033[92m{:.0f}\033[0ms'.format(time.time() - start)

            print('\r{}\t | {}\t | {}, {}\t | {}\t |'.format(fepoch, fiters, facc0, facc1, fptime), end="")
        n_iters += 1

RuntimeError: stack expects each tensor to be equal size, but got [4, 128] at entry 0 and [4, 20] at entry 1

In [11]:
print(confusion(regressor, test_loader, (0, 2)))
print(confusion(conv, test_loader, (1, 2)))

x0, x1 = np.arange(len(accuracy_histories[0])), np.arange(len(accuracy_histories[1]))
plt.plot(x0, accuracy_histories[0])
plt.plot(x1, accuracy_histories[1])

RuntimeError: stack expects each tensor to be equal size, but got [4, 20] at entry 0 and [4, 128] at entry 8